In [ ]:
%load_ext dotenv

# Use find_dotenv to locate the file
%dotenv

import os

In [ ]:
import json
import datetime

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

In [ ]:
auth=(os.getenv("ELASTICSEARCH_USER"), os.getenv("ELASTICSEARCH_PASSWORD"))

In [ ]:
transcripts_folder = "company_transcripts/"

In [ ]:
from os import listdir
from os.path import isfile, join
transcripts_files = [f for f in listdir(transcripts_folder) if isfile(join(transcripts_folder, f)) and f.split(".")[-1] == 'json']
transcripts_files[0:5]

In [ ]:
for transcripts_file in transcripts_files:
    with open(f'{transcripts_folder}/{transcripts_file}') as f:
        for row in f:
            data = json.loads(row)

            if 'title' in data and 'url' in data and 'date' in data and 'body' in data and len(data['date'])>=10 and len(data['body']) > 0 :
                final_data = {
                    'company': data['company'],
                    'title': data['title'],
                    'url': data['url'],
                    'date': data['date'][0:10],
                    'text': " ".join(data['body'])
                }
                
                query = {
                    "query": {
                        "match_phrase" : {
                            "url" : data['url']
                        }
                    }
                }
                res = requests.get(f'{os.getenv("ELASTICSEARCH_URL")}/transcripts/sa/_search', json=query, auth=auth)
                dic_res = json.loads(res.content)
                if ('error' in dic_res and dic_res['error']['root_cause'][0]['type'] == 'index_not_found_exception') or dic_res['hits']['total'] == 0:
                    res = requests.post(f'{os.getenv("ELASTICSEARCH_URL")}/transcripts/sa/', auth=auth, json=final_data)
                    print(f'{datetime.datetime.now().isoformat()} {data["company"]} {res.status_code}')
                else:
                    print(f'{datetime.datetime.now().isoformat()} {data["company"]} {data["url"]} already in db')   